In [4]:
#Caricare le librerie
import tweepy
import pandas as pd
import numpy as np
import re


In [2]:
consumer_key= "rrR0Znz98oKkjfB8b72oLH9Qc"
consumer_secret= "m7iOPX6iW8R3IalRf7x56YVeIb18o3I1bhUftg8JzlQ6fAjLgW"
access_token= "1442250607-3cJYKYyh7Ae19nMcv5YoxOIB2XlvgVX0ROaDSJx"
access_token_secret= "ZOnaaCTgO7DsKtmPjVruOlkvzBY8p6MbkKLjiYLdamBYE"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [5]:
#Mostra i miei follower
for user in tweepy.Cursor(api.get_followers, screen_name="davide_pstars").items():
  print('follower: ' + user.screen_name)

follower: kimilfung
follower: werbond
follower: a____capo
follower: SalsaConAndrea


In [6]:
#Mostra chi seguo (amici)
#Nel metodo items si può inserire un numero intero, a signifcare il numero di risultati da restituire.
for user in tweepy.Cursor(api.get_friends, screen_name="@davide_pstars").items():
  print('friend: ' + user.screen_name)

friend: reportrai3


In [7]:
# Mostro la mia timeline 
stuff = api.user_timeline(screen_name = '@davide_pstars', count = 10)

In [137]:
#Scaricare in un dataframe i tweet in base ad un hashtag (until="2021-12-21")
#NB: hasattr(obj, name, /) Return whether the object has an attribute with the given name.
hashtag="#fedez"
list_tweet=[]
list_originals=[]
n=100
for tweet in tweepy.Cursor(api.search_tweets,q=hashtag,count=100,lang="it").items(n):
  #print(tweet.text)
  #print('')
  try:
    list_originals.append(tweet.retweeted_status.user.screen_name)
  except:
    list_originals.append(None)
  list_tweet.append([tweet.created_at, tweet.id, tweet.text, 
                     tweet.favorite_count, tweet.retweet_count, 
                     tweet.user.location,tweet.user.screen_name,
                     hasattr(tweet, "retweeted_status"),
                     tweet.in_reply_to_screen_name])
tweets = pd.DataFrame(list_tweet[1:], 
                      columns=['date','id','text','like','n_rt','location','author',
                               "retweet","reply_name"])
tweets["retwitted_author"]=list_originals[1:]
tweets.head()

,date,id,text,like,n_rt,location,author,retweet,reply_name,retwitted_author
0,2021-12-18 23:10:47+00:00,1472343616978931712,#FEDEZ salta sul divano come il tizio saltava ...,0,0,,LiaCharlie1,False,None,None
1,2021-12-18 22:57:00+00:00,1472340146846638080,Chiara Ferragni e Fedez la reazione alla forte...,0,0,Italia,zazoomblog,False,None,None
2,2021-12-18 22:46:32+00:00,1472337513171886082,RT @Lanternaweb: Alcuni soggetti non hanno pro...,0,1,"Lazio, Italia",AleVerrelli,True,None,Lanternaweb
3,2021-12-18 22:20:46+00:00,1472331028148768775,La cosa più interessante de #TheFerragnezLaSer...,10,0,Roma,FabrizioDelpret,False,None,None
4,2021-12-18 22:17:23+00:00,1472330177795530755,Io voglio Leone fotografo ufficiale della nost...,0,0,"Bari, Puglia",francescomusto_,False,None,None
